In [0]:
%%sh
aws s3 ls s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/date=2021-10-02/platform=android/

In [0]:

#Fredric Query - To retreive data into a temp table view and query that table view.
df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK")
# SQL  
df.createOrReplaceTempView("raw_data")

product_1_df = spark.sql("select * from raw_data where date >='2021-02-06' and date <='2021-11-27' and device_code !='ios-all' order by product_id,date,ad_platform_id")
product_1_df.show(100, False)

In [0]:

df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/")
df.createOrReplaceTempView("localized_detail_sample")

sql = '''
    select sum(pub_est_revenue)
        from localized_detail_sample
        where product_id=20600012181548
        and country_code='MX'
        and ad_platform_id =101
        and device_code='android-all'
        and date='2021-10-30'
        GROUP BY date
'''

df_raw_locale = spark.sql(sql)

count = df_raw_locale.count() 
#df_raw_locale.show(count)

print(count)
df_raw_locale.show()

In [0]:

#Fredric Query - To connect snowflake and retreive data for comparision.

SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

SF_OPTIONS = {
    "sfURL": "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
    "sfUser": "app_bdp_data_validation_qa",
    "sfPassword": "0HN#s@Wa5$1R8jVj",
    "sfDatabase": "AA_INTELLIGENCE_PRODUCTION",
    "sfWarehouse": "wh_dod_read7"
}

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
            .options(**SF_OPTIONS) \
            .option("query", "select count(*) from AA_INTELLIGENCE_PRODUCTION.ADL_MARKET_PAID.FACT_MARKET_AD_REVENUE_V1_CLUSTER_BY_DATE where ad_revenue+revenue = total_revenue limit 3")\
            .load()
df.createOrReplaceTempView("snow_data")

df.show(10, False)



In [0]:

spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/").show()

In [0]:

df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/")
df.createOrReplaceTempView("localized_detail_sample")


In [0]:

df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/")
df.createOrReplaceTempView("localized_detail_sample")

sql = '''
    select sum(pub_est_revenue)
        from localized_detail_sample
         where product_id=20600012181548
        and country_code='MX'
        and ad_platform_id =101
        and device_code='android-all'
        and date='2021-10-30'
'''

sql2 = '''
    select S.*
        from localized_detail_sample S
        where product_id=284862083
        and country_code='US'
        and ad_platform_id =1
        --and device_code='android-all'
        and date='2021-04-10'
        order by ad_platform_id
       
'''
       


#df_raw_locale = spark.sql(sql)
df_raw_locale2= spark.sql(sql2)

#count = df_raw_locale.count() 
#df_raw_locale.show(count)

#print(count)
#df_raw_locale.show()
df_raw_locale2.show()


In [0]:


df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/range_type=WEEK/")
df.createOrReplaceTempView("localized_detail_sample")

SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

SF_OPTIONS = {
    "sfURL": "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
    "sfUser": "app_bdp_data_validation_qa",
    "sfPassword": "0HN#s@Wa5$1R8jVj",
    "sfDatabase": "AA_INTELLIGENCE_PRODUCTION",
    "sfWarehouse": "wh_dod_read7"
}

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
            .options(**SF_OPTIONS) \
            .option("query", "select r.* from AA_INTELLIGENCE_PRODUCTION.ADL_MARKET_PAID.FACT_MARKET_AD_REVENUE_V1_CLUSTER_BY_DATE r where 1=1 and GRANULARITY_CODE = 'weekly' and date>=('2021-02-06') and date<=('2021-11-27') order by r.date desc")\
            .load()
df.createOrReplaceTempView("snow_data")

sql = '''
    select S.*
        from localized_detail_sample S
        where product_id=20600013027125
        and country_code='AU'
       -- and ad_platform_id =101
        and device_code='android-all'
        and date='2021-11-13'
        order by ad_platform_id
       
'''
       


df_raw_locale = spark.sql(sql)


print(df.count())
df.show(10, False)
df_raw_locale.show(10,False)



In [0]:

#Query to check where any mismatch between the revenue in raw data and SF data.
df = spark.read.parquet("s3://aardvark-prod-dca-data/fact/MKT_AD_REVENUE/version=1.0.0/")
df.createOrReplaceTempView("localized_detail_sample")

SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

SF_OPTIONS = {
    "sfURL": "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
    "sfUser": "app_bdp_data_validation_qa",
    "sfPassword": "0HN#s@Wa5$1R8jVj",
    "sfDatabase": "AA_INTELLIGENCE_PRODUCTION",
    "sfWarehouse": "wh_dod_read7"
}

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
            .options(**SF_OPTIONS) \
            .option("query", "select r.* from AA_INTELLIGENCE_PRODUCTION.ADL_MARKET_PAID.FACT_MARKET_AD_REVENUE_V1_CLUSTER_BY_DATE r where 1=1 and GRANULARITY_CODE = 'weekly' and date>=('2021-02-06') and date<=('2021-11-27') order by r.date desc")\
            .load()
df.createOrReplaceTempView("snow_data")

#To print the total number of matching records between SF and Raw data
sqlContext.sql("select count(*) from snow_data SF,localized_detail_sample LD where SF.Product_key =LD.product_id and LD.ad_platform_id=SF.ADVERTISING_PLATFORM_KEY and LD.country_code=SF.Country_code and LD.Device_code=SF.Device_code and LD.Date=SF.Date and LD.PUB_EST_REVENUE !=SF.AD_Revenue").show()

#To print the records with Ad_Revenue which doesnt match between SF and Raw data
sqlContext.sql("select LD.PUB_EST_REVENUE,SF.AD_REVENUE,LD.Product_id,SF.Product_key,LD.ad_platform_id,SF.ADVERTISING_PLATFORM_KEY,LD.country_code,SF.Country_code,LD.Device_code,SF.Device_code,LD.Ad_type,SF.Ad_type,LD.range_type,SF.granularity_code,LD.Date,SF.Date from snow_data SF,localized_detail_sample LD where SF.Product_key =LD.product_id and LD.ad_platform_id=SF.ADVERTISING_PLATFORM_KEY and LD.country_code=SF.Country_code and LD.Device_code=SF.Device_code and LD.Ad_type=SF.Ad_type and LD.Date=SF.Date and ((LD.range_type='MONTH' and SF.granularity_code='monthly') OR (LD.range_type='WEEK' and SF.granularity_code='weekly')) and LD.PUB_EST_REVENUE !=SF.AD_Revenue").show(100,False)
